In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/NLP'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP


In [2]:
import torch
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import v_measure_score
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [3]:
embeddings = torch.load("bible_embeddings.pt")
means = torch.load("bible_similar_mean.pt")

In [4]:
data = pd.read_csv("bibledata.csv")

In [5]:
styles = embeddings - means
target = F.normalize(styles)

In [6]:
l1 = 128
l2 = 64
l3 = 32
z_dim = 16

In [21]:
class L2Norm(nn.Module):
    def __init__(self, dim=1, eps=1e-12):
        super(L2Norm, self).__init__()
        self.dim = dim
        self.eps = eps

    def forward(self, x):
        self.dim = len(x.shape) - 1
        return F.normalize(x, p=2, dim=self.dim, eps=self.eps)

In [22]:
def Encoder(x_dim, z_dim):
  model = nn.Sequential(
    nn.Linear(x_dim, l1),
    nn.ReLU(),
    nn.Linear(l1, l2),
    nn.ReLU(),
    nn.Linear(l2, l3),
    nn.ReLU(),
    nn.Linear(l3, z_dim))
  return model

In [23]:
def Decoder(x_dim, z_dim):
  model = nn.Sequential(
  nn.Linear(z_dim, l3),
    nn.ReLU(),
    nn.Linear(l3, l2),
    nn.ReLU(),
    nn.Linear(l2, l1),
    nn.ReLU(),
    nn.Linear(l1, x_dim),
    L2Norm())
  return model

In [24]:
class Model(nn.Module):
  def __init__(self, x_dim, z_dim):
      super(Model, self).__init__()
      self.Encoder = Encoder(x_dim, z_dim)
      self.Decoder = Decoder(x_dim, z_dim)
  
  def forward(self, x):
      z = self.Encoder(x)
      out = self.Decoder(z)
        
      return out

In [25]:
epochs = 30
batch_size = 64

In [12]:
dataset = TensorDataset(embeddings, target)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last = True)

In [26]:
model = Model(384, z_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [27]:
for epoch in tqdm(range(epochs)):
    overall_loss = 0
    for batch_idx, (x, label) in enumerate(train_dataloader):
        x = x.view(batch_size, 384)

        optimizer.zero_grad()

        predict = model(x)
        loss = torch.sum(torch.square(1 - F.cosine_similarity(predict, label)))
      
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    validation_loss = torch.sum(torch.square(1 - F.cosine_similarity(model(embeddings[-43:]), target[-43:]))) 
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*64), "\tValidation Loss:", validation_loss / 43)

  3%|▎         | 1/30 [00:01<00:29,  1.02s/it]

	Epoch 1 complete! 	Average Loss:  0.8754091155098145 	Validation Loss: tensor(0.7545, grad_fn=<DivBackward0>)


  7%|▋         | 2/30 [00:02<00:28,  1.01s/it]

	Epoch 2 complete! 	Average Loss:  0.7150739521147257 	Validation Loss: tensor(0.6909, grad_fn=<DivBackward0>)


 10%|█         | 3/30 [00:03<00:27,  1.03s/it]

...

	Epoch 28 complete! 	Average Loss:  0.40123374957636176 	Validation Loss: tensor(0.3716, grad_fn=<DivBackward0>)


 97%|█████████▋| 29/30 [00:36<00:01,  1.23s/it]

	Epoch 29 complete! 	Average Loss:  0.3969596814678376 	Validation Loss: tensor(0.3678, grad_fn=<DivBackward0>)


100%|██████████| 30/30 [00:37<00:00,  1.25s/it]

	Epoch 30 complete! 	Average Loss:  0.39259055167077533 	Validation Loss: tensor(0.3678, grad_fn=<DivBackward0>)


In [29]:
x0 = embeddings[10730]
y0 = target[10730]
yhat = model(x0)
torch.norm(yhat)

tensor(1.0000, grad_fn=<LinalgVectorNormBackward0>)

In [30]:
y0.dot(yhat)

tensor(0.4326, grad_fn=<DotBackward0>)

In [31]:
zs = torch.zeros((10731, z_dim))
for i in tqdm(range(len(embeddings))):
  zs[i] = model.Encoder(embeddings[i])

100%|██████████| 10731/10731 [00:03<00:00, 2741.98it/s]


In [32]:
km = KMeans(n_clusters = 7)
km.fit(zs.detach())
data["labels"] = km.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [33]:
v_measure_score(data.id, data.labels)

0.30002697504003606